In [1]:
import numpy as np
from fractions import Fraction

In [50]:
small_data_dim = [(7,5), (10,5), (15,10),(15,5),(15,10)]
small_data_intervals = [(-10, 10), (-20, 20), (-30, 30), (-40, 40), (-50, 50)]
big_data_dim = [(35,30), (45,30),(150,50), (150,100), (200,150)]
big_data_intervals = [(-150, 150), (-200,200), (-250,250), (-300,300), (-350,350)]

In [51]:
for i in range(len(small_data_dim)):
    m = small_data_dim[i][0]
    n = small_data_dim[i][1]
    a = small_data_intervals[i][0]
    b = small_data_intervals[i][1]
    A = np.random.randint(a, b, size=(m,n))
    b = np.random.randint(a,b, size=m)
    try:
        x = np.linalg.lstsq(A, b, rcond=None)[0]
    except np.linalg.LinAlgError:
        print('nema resenje')
        i -= 1;
    np.save(f'data/small_data/{i}_A', A)
    np.save(f'data/small_data/{i}_b', b)
    

In [52]:
for i,((m,n),(a,b)) in enumerate(zip(big_data_dim, big_data_intervals)):
    A = np.random.randint(a, b, size=(m,n))
    b = np.random.randint(a,b, size=m)
    np.save(f'data/big_data/{i}_A', A)
    np.save(f'data/big_data/{i}_b', b)